In [74]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Literal
from pydantic import BaseModel, Field
import operator

In [75]:
load_dotenv()

True

In [76]:
generator_llm = ChatOpenAI(model_name="gpt-4o-mini")
evaluator_llm = ChatOpenAI(model_name="gpt-4o-mini")
optimizer_llm = ChatOpenAI(model_name="gpt-4.1")

In [77]:
class TweetState(TypedDict):
    topic : str
    tweet : str
    evaluation : Literal["approved","needs_improvement"]
    feedback : str
    iteration : int
    max_iteration : int
    tweet_history : Annotated[list[str], operator.add]
    feedback_history : Annotated[list[str], operator.add]


In [78]:
def generate_tweet(state : TweetState):
    messages = [
        SystemMessage(content="You are a funny and clever Tweeter/X influencer."),
        HumanMessage(content = f"""write a short, original and hilarious tweet on the topic: {state['topic']} 
        
        Rules :
        - Do not use question-answer format.
        - Max 280 characters.
        - Use observational humor, irony, sarcasm or cultural references.
        - Think in meme logic, punchlines or relatable takes.
        - Use simple, day to day english.""")
    ]
    
    response = generator_llm.invoke(messages).content
    return {"tweet":response, "tweet_history" : [response]}

In [79]:
class tweet_schema(BaseModel):
    evaluation : Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback : str = Field(..., description="feedback for the tweet")
    

In [80]:
structured_evaluator_llm = evaluator_llm.with_structured_output(tweet_schema)

In [81]:
def evaluate_tweet(state: TweetState):
    messages = [
        SystemMessage(
            content=(
                "You are a ruthless, no-laugh-given Twitter critic. "
                "You evaluate tweets based on humour, originality, virality, and tweet format."
            )
        ),
        HumanMessage(
            content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality: Is this fresh or have you seen it a hundred times before?
2. Humour: Did it genuinely make you smile, laugh, or chuckle?
3. Punchiness: Is it short, sharp, and scroll-stopping?
4. Virality Potential: Would people retweet or share it?
5. Format: Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters.)

Auto-reject if:
- It's written in question-answer format (e.g., "why did..." or "what happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke.
- It ends with generic, throwaway, or deflating lines that weaken the humour (e.g., "Masterpieces of the auntie-uncle universe" or vague summaries)

## Respond only in structured format:
- evaluation: "approved" or "needs_improvement"
- feedback: one paragraph explaining the strengths and weaknesses
"""
        )
    ]

    response = structured_evaluator_llm.invoke(messages)
    return {
        "evaluation": response.evaluation,
        "feedback": response.feedback,
        "feedback_history" : [response.feedback]
    }


In [82]:
def optimize_tweet(state : TweetState):
    messages = [
        SystemMessage(content="You punch up tweets for virality and humour based on given feedback"),
        HumanMessage(content=f"""Improve the tweet based on this feedback : "{state['topic']}"
        Topic: "{state['topic']}"
        Original Tweet : {state['tweet']}

        Re-write it as a short , viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
        """)
    ]

    response = optimizer_llm.invoke(messages).content
    iteration = state['iteration'] + 1
    return {"tweet":response, "iteration" : iteration , "tweet_history" : [response]}

In [83]:
def route_evaluation(state : TweetState):
    if state['evaluation'] == "approved" or state['iteration'] >= state['max_iteration']:
        return "approved"
    else :
        return "needs_improvement"

In [84]:
graph = StateGraph(TweetState)

graph.add_node("generate_tweet", generate_tweet)
graph.add_node("evaluate_tweet", evaluate_tweet)
graph.add_node("optimize_tweet", optimize_tweet)

graph.add_edge(START, "generate_tweet")
graph.add_edge("generate_tweet","evaluate_tweet")
graph.add_conditional_edges("evaluate_tweet", route_evaluation, {"approved":END, "needs_improvement":"optimize_tweet"})
graph.add_edge("optimize_tweet", "evaluate_tweet")

workflow = graph.compile()

In [85]:
initial_state = {
    "topic": "ABCDEFG",
    "iteration": 0,
    "max_iteration": 5
}

workflow.invoke(initial_state)

{'topic': 'ABCDEFG',
 'tweet': 'A, B, and C are out here running the alphabet clique while Z is chilling in the back like it’s waiting for the encore. H, I, and J just hitching a ride for popularity points. #AlphabetDrama #ZInTheBack',
 'evaluation': 'approved',
 'feedback': 'This tweet exhibits strong originality by personifying letters in the alphabet and crafting a humorous narrative around them. The creativity shines through as it cleverly paints a scenario where Z is the overlooked member, which is both amusing and relatable. The punchiness is adequate, making it engaging enough to stop users from scrolling, while the hashtags add to its potential for virality. The format is solid, keeping within the character limit and avoiding tired joke structures. Overall, it effectively combines humor and creativity, making it shareable.',
 'iteration': 2,
 'max_iteration': 5,
 'tweet_history': ['Just finished the alphabet, and honestly, I don’t know why H, I, and J thought they could just ha